# Viola Jones

Detekcja wybranych zdjęć ze zbioru danych

In [1]:
import cv2
def viola_detect_on_img(filename):
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    img = cv2.imread(filename)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    for (x,y,w,h) in faces:
        img = cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
    cv2.namedWindow('Face Detection')
    cv2.imshow('Face Detection', img)
    if cv2.waitKey(0):
        cv2.destroyAllWindows()

In [3]:
file1 = 'WIDER_train\images/35--Basketball/35_Basketball_Basketball_35_98.jpg'
viola_detect_on_img(file1)

In [4]:
file2 = 'WIDER_train/images/56--Voter/56_Voter_peoplevoting_56_48.jpg'
viola_detect_on_img(file2)

Detekcja na wideo

In [33]:
def viola_detect_on_camera(save_to_avi=False):
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    cap = cv2.VideoCapture(0)
    success, frame = cap.read()

    if save_to_avi:
        
        size = (int(cap.get(3)), int(cap.get(4)))
                # Below VideoWriter object will create a frame of above defined. The output is stored in 'filename.avi' file.
        result = cv2.VideoWriter('results/viola.avi', 
                                cv2.VideoWriter_fourcc(*'MJPG'),
                                10, size)
            
        while success:

            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            faces = face_cascade.detectMultiScale(gray, 1.3, 5)
            for (x,y,w,h) in faces:
                cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
            
            # Write the frame into the
            # file 'filename.avi'
            result.write(frame)
    
            # Display the frame
            # saved in the file
            cv2.imshow('Frame', frame)
    
            # Press S on keyboard 
            # to stop the process
            if cv2.waitKey(1) & 0xFF == ord('s'):
                cap.release()
                result.release()                    
                # Closes all the frames
                cv2.destroyAllWindows()
                break
            success, frame = cap.read()

    else:
        while success:
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            faces = face_cascade.detectMultiScale(gray, 1.3, 5)
            for (x,y,w,h) in faces:
                cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)

            cv2.imshow("camera", frame)
            if cv2.waitKey(1) == 27: # Escape
                cv2.destroyAllWindows()
                break
            success, frame = cap.read()

In [34]:
viola_detect_on_camera(save_to_avi=True)

# SSD

https://github.com/weiliu89/caffe/tree/ssd/

https://github.com/PacktPublishing/Learning-OpenCV-4-Computer-Vision-with-Python-Third-Edition/blob/master/chapter10/faces_dnn.py

From book:

"Learning-OpenCV-4-Computer-Vision-With-Python-3-Get-to-Grips-With-Tools-Techniques-And-Algorithms-for-Computer-Vision-and-Machine-Learning-by-Joseph"

Detekcja wybranych zdjęć ze zbioru danych

In [7]:
import cv2
import numpy as np

face_model = cv2.dnn.readNetFromCaffe(
    'ssd/deploy.prototxt',
    'ssd/res10_300x300_ssd_iter_140000.caffemodel')

In [41]:
def ssd_detect_on_img(ssd_model, filename):

    img = cv2.imread(filename)
    face_model = ssd_model
    face_blob_height = 300
    face_average_color = (104, 177, 123)
    face_confidence_threshold = 0.95

    while not img is None:
        h, w = img.shape[:2]
        aspect_ratio = w/h

        # Detect faces in the frame.

        face_blob_width = int(face_blob_height * aspect_ratio)
        face_blob_size = (face_blob_width, face_blob_height)

        face_blob = cv2.dnn.blobFromImage(
            img, size=face_blob_size, mean=face_average_color)

        face_model.setInput(face_blob)

        try:        
            face_results = face_model.forward()

            # Iterate over the detected faces.
            for face in face_results[0, 0]:
                face_confidence = face[2]
                if face_confidence > face_confidence_threshold:

                    # Get the face coordinates.
                    x0, y0, x1, y1 = (face[3:7] * [w, h, w, h]).astype(int)

                    # Draw a blue rectangle around the face.
                    cv2.rectangle(img, (x0, y0), (x1, y1),
                                    (255, 0, 0), 2)

            cv2.imshow('Faces', img)

            if cv2.waitKey(1) == 27: # Escape
                cv2.destroyAllWindows()
                break
        except:
            print("No face detected on the image")
            break

In [40]:
file1 = 'WIDER_train/images/8--Election_Campain/8_Election_Campain_Election_Campaign_8_15.jpg'
# file1 = 'WIDER_train\images/35--Basketball/35_Basketball_Basketball_35_98.jpg'
ssd_detect_on_img(face_model, file1)

Error, no face detected


In [42]:
file2 = 'WIDER_train/images/56--Voter/56_Voter_peoplevoting_56_48.jpg'
ssd_detect_on_img(face_model ,file2)

Detekcja na wideo

In [20]:
import cv2
import numpy as np

face_model = cv2.dnn.readNetFromCaffe(
    'ssd/deploy.prototxt',
    'ssd/res10_300x300_ssd_iter_140000.caffemodel')

In [22]:
def ssd_detect_on_camera(ssd_model, save_to_avi=False):

    face_blob_height = 300
    face_average_color = (104, 177, 123)
    face_confidence_threshold = 0.9

    cap = cv2.VideoCapture(0)

    success, frame = cap.read()

    if save_to_avi:
        
        size = (int(cap.get(3)), int(cap.get(4)))
                # Below VideoWriter object will create a frame of above defined. The output is stored in 'filename.avi' file.
        result = cv2.VideoWriter('results/ssd.avi', 
                                cv2.VideoWriter_fourcc(*'MJPG'),
                                10, size)
        while success:

            h, w = frame.shape[:2]
            aspect_ratio = w/h

            # Detect faces in the frame.

            face_blob_width = int(face_blob_height * aspect_ratio)
            face_blob_size = (face_blob_width, face_blob_height)

            face_blob = cv2.dnn.blobFromImage(
                frame, size=face_blob_size, mean=face_average_color)

            face_model.setInput(face_blob)
            face_results = face_model.forward()
            # Iterate over the detected faces.
            for face in face_results[0, 0]:
                face_confidence = face[2]
                if face_confidence > face_confidence_threshold:

                    # Get the face coordinates.
                    x0, y0, x1, y1 = (face[3:7] * [w, h, w, h]).astype(int)

                    

                    # Draw a blue rectangle around the face.
                    cv2.rectangle(frame, (x0, y0), (x1, y1),
                                (255, 0, 0), 2)

                # Write the frame into the
            # file 'filename.avi'
            result.write(frame)
    
            # Display the frame
            # saved in the file
            cv2.imshow('Frame', frame)
    
            # Press S on keyboard 
            # to stop the process
            if cv2.waitKey(1) & 0xFF == ord('s'):
                cap.release()
                result.release()                    
                # Closes all the frames
                cv2.destroyAllWindows()
                break
            success, frame = cap.read()


    else:

        while success:

            h, w = frame.shape[:2]
            aspect_ratio = w/h

            # Detect faces in the frame.

            face_blob_width = int(face_blob_height * aspect_ratio)
            face_blob_size = (face_blob_width, face_blob_height)

            face_blob = cv2.dnn.blobFromImage(
                frame, size=face_blob_size, mean=face_average_color)

            face_model.setInput(face_blob)
            face_results = face_model.forward()
            # Iterate over the detected faces.
            for face in face_results[0, 0]:
                face_confidence = face[2]
                if face_confidence > face_confidence_threshold:

                    # Get the face coordinates.
                    x0, y0, x1, y1 = (face[3:7] * [w, h, w, h]).astype(int)

                    

                    # Draw a blue rectangle around the face.
                    cv2.rectangle(frame, (x0, y0), (x1, y1),
                                (255, 0, 0), 2)

            cv2.imshow('Faces', frame)
            if cv2.waitKey(1) == 27: # Escape
                cv2.destroyAllWindows()
                break

            success, frame = cap.read()

In [35]:
ssd_detect_on_camera(face_model, save_to_avi=True)

# MTCNN

Detekcja wybranych zdjęć ze zbioru danych

In [24]:
import cv2
from mtcnn.mtcnn import MTCNN
mtcnn_detector = MTCNN()

In [7]:
def mtcnn_detect_on_img(mtcnn_model, filename):
    img = cv2.imread(filename)

    while not img is None:

        faces = mtcnn_model.detect_faces(img)# result
        #to draw faces on image
        for result in faces:
            x, y, w, h = result['box']
            x1, y1 = x + w, y + h
            cv2.rectangle(img, (x, y), (x1, y1), (0, 0, 255), 2)

        cv2.imshow('face detected', img)
        if cv2.waitKey(1) == 27: # Escape
            cv2.destroyAllWindows()
            break

In [11]:
# file1 = 'WIDER_train/images/8--Election_Campain/8_Election_Campain_Election_Campaign_8_15.jpg'
file1 = 'WIDER_train\images/35--Basketball/35_Basketball_Basketball_35_98.jpg'
mtcnn_detect_on_img(mtcnn_detector, file1)

1/1 [==============================] - 0s 35ms/step


In [ ]:
file2 = 'WIDER_train/images/56--Voter/56_Voter_peoplevoting_56_48.jpg'
mtcnn_detect_on_img(mtcnn_detector ,file2)

Detekcja na wideo

In [28]:
def mtcnn_detect_on_camera(mtcnn_model, save_to_avi=True):

    cap = cv2.VideoCapture(0)
    success, frame = cap.read()

    if save_to_avi:
        size = (int(cap.get(3)), int(cap.get(4)))
                # Below VideoWriter object will create a frame of above defined. The output is stored in 'filename.avi' file.
        result = cv2.VideoWriter('results/mtcnn.avi', 
                                cv2.VideoWriter_fourcc(*'MJPG'),
                                10, size)
        
        while success:

            faces = mtcnn_model.detect_faces(frame)# result
            if faces != []:
                for face in faces:
                    traced_rectangle = face['box']
                    cv2.rectangle(frame,(traced_rectangle[0], traced_rectangle[1]),
                    (traced_rectangle[0] + traced_rectangle[2],
                    traced_rectangle[1] + traced_rectangle[3]),(0,155,255),2)

            # Write the frame 
            result.write(frame)
    
            # Display the frame
            # saved in the file
            cv2.imshow('Frame', frame)
    
            # Press S on keyboard 
            # to stop the process
            if cv2.waitKey(1) & 0xFF == ord('s'):
                cap.release()
                result.release()                    
                # Closes all the frames
                cv2.destroyAllWindows()
                break
            success, frame = cap.read()

    else:
        while success:

            faces = mtcnn_model.detect_faces(frame)# result
            if faces != []:
                for face in faces:
                    traced_rectangle = face['box']
                    cv2.rectangle(frame,(traced_rectangle[0], traced_rectangle[1]),
                    (traced_rectangle[0] + traced_rectangle[2],
                    traced_rectangle[1] + traced_rectangle[3]),(0,155,255),2)

            cv2.imshow('face detected', frame)
            if cv2.waitKey(1) == 27: # Escape
                cv2.destroyAllWindows()
                break
        
            success, frame = cap.read()

In [36]:
mtcnn_detect_on_camera(mtcnn_detector, save_to_avi=True)

1/1 [==============================] - 0s 54ms/step


In [ ]:
def mtcnn_detect_on_camera(mtcnn_model):

    cap = cv2.VideoCapture(0)
    success, frame = cap.read()

    while success:

        faces = mtcnn_model.detect_faces(frame)# result
        if faces != []:
            for face in faces:
                traced_rectangle = face['box']
                cv2.rectangle(frame,(traced_rectangle[0], traced_rectangle[1]),
                (traced_rectangle[0] + traced_rectangle[2],
                traced_rectangle[1] + traced_rectangle[3]),(0,155,255),2)

        cv2.imshow('face detected', frame)
        if cv2.waitKey(1) == 27: # Escape
            cv2.destroyAllWindows()
            break
    
        success, frame = cap.read()

In [29]:
mtcnn_detect_on_camera(mtcnn_detector)

1/1 [==============================] - 0s 32ms/step


In [27]:
capture = cv2.VideoCapture(0)
while True:
        # Continuous capture of the video feed
        __, frame = capture.read()
        ''' Create an instance of the MTCNN() function/class, using default
            weights. '''
        detector = MTCNN()
        # This internal method detects faces in the video feed
        result = detector.detect_faces(frame)
        if result != []:
            for person in result:
                traced_rectangle = person['box']
                # Trace the rectangle encasing the faces 
                cv2.rectangle(frame,(traced_rectangle[0], traced_rectangle[1]),
                              (traced_rectangle[0] + traced_rectangle[2],
                               traced_rectangle[1] + traced_rectangle[3]),(0,155,255),2)
                
            # Show the tracing in the video feed
            cv2.imshow('frame',frame)
            # Stop the video capture if 'q' is pressed
            if cv2.waitKey(1) == 27: #ESC
                capture.release()
                cv2.destroyAllWindows()
                break

1/1 [==============================] - 0s 273ms/step
